<h1 style=font-family: "Times New Roman", Times, serif;>PROJETO ENGG51 - TRANSFORMAÇÃO DE ÁUDIOS</h1>

<h3> Intalação de bibliotecas necessárias</h3>

In [ ]:
!pip install pydub
!pip install numpy
!pip install matplotlib
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install gspread
#Bibliotecas sys e os já existem no sistema windows

<h3>Etapa de carregamento de arquivos e indexação</h3>

In [ ]:
#Importando bibliotecas
import os
from pydub import AudioSegment

#Busca o diretório de trabalho
diretorio = ('C:\\Users\\JuuHH\\Documents\\Projeto ENGG51')

#Define vetores
nomeInicial = []
nomeAlterado = []
nomeFinal = []

#ETAPA 1 - Indexação numérica de arquivos
i=0
for arquivo in os.listdir(diretorio):                                           #Busca arquivos no diretório
    if (arquivo).endswith('.wav'):
        i+=1 
        
        antigoNome = diretorio+'\\'+arquivo
        novoNome = diretorio+'\\'+str(i)+'.wav'
        
        os.rename(antigoNome,novoNome)                                           #Renomeia
        
        nomeInicial.append(arquivo)
        nomeAlterado.append(str(i)+'.wav')
        
print("Os arquivos ", nomeInicial, " foram renomeados para ", nomeAlterado)      #Resultados da etapa

#ETAPA 2 - Conversão de arquivos de stereo para mono
i = 0
for arquivo in os.listdir(diretorio):                              #Busca arquivos no diretório
    if (arquivo).endswith('.wav'):
        i+=1
        nome = str(i)+'_mono'+'.wav'
        sound = AudioSegment.from_wav(diretorio+'\\'+arquivo)      #seleção do arquivo para conversão
        sound = sound.set_channels(1)                              #setagem de canais do arquivo
        sound.export(diretorio+'\\'+nome, format="wav")            #exportação do arquivo convertido
        
        nomeFinal.append(nome)                                     #armazena nome dos arquivos mono
        
        
print("Os seguintes arquivos de áudio mono foram gerados: \n", nomeFinal) 



<h3>Etapa de plotagem de gráfico</h3>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import wave, sys 

#exemplo de endereço => 'C:\\Users\\JuuHH\\Documents\\Projeto ENGG51\\1_mono.wav'

print("Entre com o endereço do arquivo que deseja gerar o gráfico: \n")
endereco = input()

#ETAPA 3 - plota o eixo x em segundos usando a taxa de quadros

def visualize(path: str): 
    
    raw = wave.open(endereco)                                         #Abre arquivos de áudio

    signal = raw.readframes(-1)                                       #Lê todos os frames da onda sonora aberta
    signal = np.frombuffer(signal, dtype ="int16")                    

    f_rate = raw.getframerate()                                       #Armazena a taxa de quadros em uma variável

    time = np.linspace(0, len(signal) / f_rate, num = len(signal))    

#Traça o gráfico derivado 

    plt.figure(1)                                                                  
    plt.title("Sound Wave") 
    plt.xlabel("Time") 

    plt.plot(time, signal)                                            

    plt.show() 

    
if __name__ == "__main__": 
    path = sys.argv[1] 
    visualize(endereco) 

<h3>Escrita em Planilha Google</h3>

In [ ]:
import time
import numpy as np
import wave, sys 
import gspread
import itertools 
from google.oauth2 import service_account
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

#exemplo de endereço => 'C:\\Users\\JuuHH\\Documents\\Projeto ENGG51\\1_mono.wav'

# permissoes pra gerenciar a planilha
scopes = ["https://www.googleapis.com/auth/spreadsheets",
          "https://www.googleapis.com/auth/drive"]

# arquivo que contem informacoes da conta de servico          
key_file = 'C:\\Users\\JuuHH\\JN Projects\\key.json'

infosAudio = []

print("Entre com o endereço do arquivo que deseja transcrever os dados: \n")
endereco = input()

# recolhe informacoes dos audios escolhidos 
def relatorio(endereco: str): 
    with wave.open(endereco) as wf:
        infosAudio.append(endereco.split("\\")[10])                               #Nome do arquivo
        infosAudio.append(wf.getnchannels())                                      #Número de canais
        infosAudio.append(wf.getnframes())                                        #Número de quadros
        infosAudio.append(wf.getframerate())                                      #Taxa de quadros
        infosAudio.append(wf.getsampwidth())                                      #Largura das amostras
        infosAudio.append(round(wf.getnframes() / wf.getframerate(), 2))          #duração do audio em segundos
        print(' Name: ', endereco.split("\\")[10], '\n', 'Channels: ', wf.getnchannels(), '\n', 'Frames: ', wf.getnframes(), '\n', 'Frame rate: ', wf.getframerate(), 'Hz', '\n', 'Samplewidth: ', wf.getsampwidth(), 'bytes', '\n', 'Duration: ', round(wf.getnframes() / wf.getframerate(), 2), 'seconds')
    return infosAudio
        
relatorio(endereco) 


# é feito login com a conta de servico para acessar a planilha
def login():
    credentials = service_account.Credentials.from_service_account_file(key_file)
    scoped_credentials = credentials.with_scopes(scopes)
    gc = gspread.authorize(scoped_credentials)
    return gc

# identifica qual linha está vazia na planilha
def linhaVazia(planilha):
    strList = list(filter(None, planilha.col_values(1)))  #retorna uma lista com as palavras da coluna 1
    return (len(strList)+1)


#escreve dados na planilha
def escritor(title,info):
    gc = login()
    planilha = gc.open('Audios Transformados_engg51')
    planilha = planilha.worksheet('Página 1')
    
    proximaLinha = linhaVazia(planilha)
    refc1 = "A{}".format(proximaLinha)
    refc2 = "B{}".format(proximaLinha)
    
    planilha.append_row([title], table_range=refc1)
    time.sleep(1)
    planilha.append_row([info], table_range=refc2)
    

titles = ['Name', 'Channels', 'Frames', 'Frame rate', 'Sample width', 'Duration']

for (title, info) in zip(titles, infosAudio):
    i=i+1
    escritor(title, info)

escritor(" ", " ")